In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
import keras
print(keras.__version__)

2024-08-14 02:08:08.626461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 02:08:08.626564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 02:08:08.752441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


3.4.1


In [4]:
!pip install --upgrade --no-cache-dir gdown
!gdown 1jIl8uXsrdRz3ZBxljjQNo0ssDT8LbJBx

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Downloading...
From (original): https://drive.google.com/uc?id=1jIl8uXsrdRz3ZBxljjQNo0ssDT8LbJBx
From (redirected): https://drive.google.com/uc?id=1jIl8uXsrdRz3ZBxljjQNo0ssDT8LbJBx&confirm=t&uuid=37d8fcf3-fcc8-4aa0-80f3-8de041dc654f
To: /kaggle/working/will_not_travel_again_data.zip
100%|████████████████████████████████████████| 188M/188M [00:23<00:00, 8.13MB/s]


In [6]:
!unzip /kaggle/working/will_not_travel_again_data.zip

Archive:  /kaggle/working/will_not_travel_again_data.zip
   creating: data/
  inflating: data/test.csv           
  inflating: data/train.csv          


In [7]:
train = pd.read_csv('/kaggle/working/data/train.csv', engine='pyarrow')
test = pd.read_csv('/kaggle/working/data/test.csv', engine='pyarrow')

train.head()

,user,user_location_country,user_location_region,user_location_city,destination_distance,search_date,is_mobile,is_package,channel,search_count,...,n_adults,n_children,n_rooms,destination,destination_type,hotel_continent,hotel_country,hotel_market,hotel_category,is_booking
0,461899,3,50,5703,NaN,2013-01-07 00:00:02,0,0,9,1,...,2,1,1,669,3,2,50,212,41,0
1,13796,66,174,21177,5713.6206,2013-01-07 00:00:06,0,0,9,3,...,1,0,1,8821,1,6,17,30,58,0
2,1128575,205,155,14703,795.7298,2013-01-07 00:00:06,0,0,9,1,...,1,0,1,25064,6,2,50,1230,91,0
3,1080476,69,761,41949,NaN,2013-01-07 00:00:17,0,1,9,1,...,2,0,1,7635,3,2,50,675,10,0
4,1080476,69,761,41949,NaN,2013-01-07 00:00:23,0,1,9,1,...,2,0,1,7635,3,2,50,675,10,0


In [8]:
train = train.drop(columns='user')
test = test.drop(columns=['user', ''])

In [9]:
train = train.dropna(subset=['checkIn_date', 'checkOut_date'])
test = test.dropna(subset=['checkIn_date', 'checkOut_date'])

In [10]:
train.isna().mean()

user_location_country    0.000000
user_location_region     0.000000
user_location_city       0.000000
destination_distance     0.357627
search_date              0.000000
is_mobile                0.000000
is_package               0.000000
channel                  0.000000
search_count             0.000000
checkIn_date             0.000000
checkOut_date            0.000000
n_adults                 0.000000
n_children               0.000000
n_rooms                  0.000000
destination              0.000000
destination_type         0.000000
hotel_continent          0.000000
hotel_country            0.000000
hotel_market             0.000000
hotel_category           0.000000
is_booking               0.000000
dtype: float64

In [ ]:
mean = train.groupby(['destination', 'user_location_city'])['destination_distance'].mean().to_dict()

def get_mean_value(row, mean_dict):
    return mean_dict.get((row['destination'], row['user_location_city']), None)

train['mean_destination_distance'] = train.apply(lambda row: get_mean_value(row, mean), axis=1)
test['mean_destination_distance'] = test.apply(lambda row: get_mean_value(row, mean), axis=1)
train['destination_distance'] = train['destination_distance'].fillna(train['mean_destination_distance'])
test['destination_distance'] = test['destination_distance'].fillna(test['mean_destination_distance'])
train.drop(columns=['mean_destination_distance'], inplace=True)
test.drop(columns=['mean_destination_distance'], inplace=True)

In [ ]:
train.isna().sum()

In [ ]:
train[train['destination_distance'] == 0]

In [ ]:
train['destination_distance'] = train['destination_distance'].fillna(0) #TODO
test['destination_distance'] = test['destination_distance'].fillna(0) #TODO

In [ ]:
train.isna().sum()

In [ ]:
time_columns = ['search_date', 'checkIn_date', 'checkOut_date']

for col in time_columns:
    train[col] = pd.to_datetime(train[col])
    test[col] = pd.to_datetime(test[col])

In [ ]:
duration = (train['checkOut_date'] - train['checkIn_date']).dt.days
train['duration'] = duration
duration = (test['checkOut_date'] - test['checkIn_date']).dt.days
test['duration'] = duration
del duration

days_between = (train['checkIn_date'] - train['search_date']).dt.days
train['days_between'] = days_between
days_between = (test['checkIn_date'] - test['search_date']).dt.days
test['days_between'] = days_between
del days_between

In [ ]:
train['search_date_hour'] = train['search_date'].dt.hour
train['search_date_dayofweek'] = train['search_date'].dt.dayofweek
train['checkIn_date_dayofweek'] = train['checkIn_date'].dt.dayofweek
train['search_date_month'] = train['search_date'].dt.month
train['checkIn_date_month'] = train['checkIn_date'].dt.month

test['search_date_hour'] = test['search_date'].dt.hour
test['search_date_dayofweek'] = test['search_date'].dt.dayofweek
test['checkIn_date_dayofweek'] = test['checkIn_date'].dt.dayofweek
test['search_date_month'] = test['search_date'].dt.month
test['checkIn_date_month'] = test['checkIn_date'].dt.month

In [ ]:
is_b = train[(train['is_booking'] == 1)]
not_b = train[(train['is_booking'] == 0)]

In [ ]:
import plotly.graph_objects as go

trace_not_booked = go.Bar(y = not_b.groupby('search_date_hour').size() / len(not_b) , name='Not Booked') # TODO
trace_is_booked = go.Bar(y = is_b.groupby('search_date_hour').size() / len(is_b) , name='Booked') # TODO

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Search Hour', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./search_hour.json')

In [ ]:
train['search_is_morning'] = np.where((6<=train['search_date_hour'])&(train['search_date_hour']<=12), 1, 0)
test['search_is_morning'] = np.where((6<=test['search_date_hour'])&(test['search_date_hour']<=12), 1, 0)

In [ ]:
trace_not_booked = go.Bar(y=not_b.groupby('checkIn_date_dayofweek').size() / len(train) , name='Not Booked') # TODO
trace_is_booked = go.Bar(y=is_b.groupby('checkIn_date_dayofweek').size() / len(train) , name='Booked') # TODO

ticktext = [
    'دوشنبه',
    'سه‌شنبه',
    'چهارشنبه',
    'پنج‌شنبه',
    'جمعه',
    'شنبه',
    'یکشنبه',
]

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Day of Week', tickangle=45, automargin=True,
               tickvals = [0,1,2,3,4,5,6], ticktext= ticktext
 ),
    yaxis=dict(title='Frequency'),
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./checkIn_day.json')

In [ ]:
trace_not_booked = go.Bar(y = not_b.groupby('checkIn_date_month').size() / len(train) , name='Not Booked') # TODO
trace_is_booked = go.Bar(y = is_b.groupby('checkIn_date_month').size() / len(train) , name='Booked') # TODO

data = [trace_is_booked, trace_not_booked]

ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

layout = go.Layout(
    xaxis=dict(title='Month', tickangle=45, automargin=True,
             ticktext = ticktext ,tickvals = np.arange(0,12)),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./checkIn_date_month.json')

In [ ]:
trace_not_booked = go.Scatter(y = not_b.groupby('days_between').size() / len(train) , name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y = is_b.groupby('days_between').size() / len(train) , name='Booked') # TODO

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Days between search and checking time', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./days_between.json')

In [ ]:
trace_not_booked = go.Scatter(y = not_b.groupby('duration').size() / len(train) , name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y = is_b.groupby('duration').size() / len(train) , name='Booked') # TODO

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Length of Stay', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./los.json')

In [ ]:
train['is_abroad'] = np.where(train['user_location_country'] != train['hotel_country'], 1, 0)
test['is_abroad'] = np.where(test['user_location_country'] != test['hotel_country'], 1, 0)

In [ ]:
train.columns

In [ ]:
col_rem = ['user_location_country', 'user_location_city', 'search_date', 'is_mobile',
          'checkIn_date', 'checkOut_date', 'hotel_continent']
train = train.drop(columns=col_rem)
test = test.drop(columns=col_rem)

In [ ]:
for i in train.columns:
    print(f'{i} : {len(train[i].unique())}')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

for i in test.columns:
    scaler = MinMaxScaler()
    train[i] = scaler.fit_transform(train[[i]])
    test[i] = scaler.transform(test[[i]])

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(columns='is_booking')
y = train['is_booking']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2)
X_val, X_test, y_val, y_test = train_test_split(X_valid, y_valid, test_size=.5)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=(X_train.shape[1], 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation='elu'),
    keras.layers.Dropout(.3),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(50, activation='elu'),
    keras.layers.Dropout(.3),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(50, activation='elu'),
    keras.layers.Dropout(.3),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(1, activation='sigmoid'),
])

In [ ]:
initial_learning_rate = .01
model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate),
             loss='binary_crossentropy',
             metrics=['accuracy'],
             )

In [ ]:
epochs = 100
BATCH_SIZE = 4096

checkpoint_cb = keras.callbacks.ModelCheckpoint('best_mcp.keras', save_best_only=True,
                                                monitor='val_loss', mode='min', verbose=1)

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=BATCH_SIZE,
                    validation_data=[X_val, y_val],
                    callbacks=[checkpoint_cb]
                   ) #TODO

In [ ]:
from sklearn.metrics import roc_auc_score

pred = model.predict(X_valid)
pred = np.where(pred >= .5, 1, 0)
roc_auc_score(pred, y_valid)

In [ ]:
pred = model.predict(test).flatten()
submission = pd.DataFrame({'prediction': pred})

In [ ]:
submission.head()